### Librerias necesarias para este modelo 


In [1]:
import imageio 
import numpy as np
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import os 
from glob import glob
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import v2
import cv2
import torchvision.transforms as transforms
from skimage.metrics import mean_squared_error as MSE
from skimage.metrics import structural_similarity as ssim
import wandb
import matplotlib.animation as animation
from PIL import Image
from collections import defaultdict

### Funciones de utilidad para el modelo

In [2]:
def saveImage(filename, image):
    imageTMP = np.clip(image * 255.0, 0, 255).astype('uint8')
    imageio.imwrite(filename, imageTMP)


In [3]:
def calculate_mse(img1, img2):
    return torch.mean((img1 - img2) ** 2)

In [4]:
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self, images_list):
        
        self.imgs = sorted(images_list)

    def __len__(self):

        return len(self.imgs)

    def __getitem__(self, idx):

        img_path = self.imgs[idx]

        image = cv2.imread(img_path)
        if image is None:
            raise ValueError(f"No se pudo cargar la imagen en la ruta: {img_path}")

        image = (image / 255.0).astype(np.float32)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        assert image.shape[-1] == 3, "La imagen no tiene 3 canales."

        img_tensor = torch.from_numpy(image).permute(2, 0, 1)

        return img_tensor

Creamos el dataloader de train


In [5]:
PATHc="/home/brulon/Documentos/TFG/entrenamiento/5"    
IMGS_PATHc=sorted(glob(os.path.join(PATHc, "*.jpg"))) 

In [ ]:
   
batchsize = 4  

dataset = MyDataset(IMGS_PATHc)

dataloader = DataLoader(dataset, batch_size=batchsize, num_workers=4, drop_last=True, shuffle=True)   

dataloader


### La red neuronal: U-net y el definición del modelo 

Se podría sustituir por otra pero en este caso usamos una simple y util, U-net, localizamos el modelo en la variable D y definimos el optimizador a ADAM

In [6]:
class Unet(torch.nn.Module):
    def __init__(self):
        super(Unet, self).__init__()

        # block down 1
        self.block1_conv1 = torch.nn.Conv2d( 6, 64, kernel_size=(3,3), padding=(1,1), padding_mode='zeros', stride=1)  #numero de canales es 6 
        self.block1_conv2 = torch.nn.Conv2d(64, 64, kernel_size=(3,3), padding=(1,1), padding_mode='zeros', stride=2)
        # block down 2
        self.block2_conv1 = torch.nn.Conv2d(64, 64, kernel_size=(3,3), padding=(1,1), padding_mode='zeros', stride=1)
        self.block2_conv2 = torch.nn.Conv2d(64, 64, kernel_size=(3,3), padding=(1,1), padding_mode='zeros', stride=2)
        # block down 3
        self.block3_conv1 = torch.nn.Conv2d(64, 64, kernel_size=(3,3), padding=(1,1), padding_mode='zeros', stride=1)
        self.block3_conv2 = torch.nn.Conv2d(64, 64, kernel_size=(3,3), padding=(1,1), padding_mode='zeros', stride=1)
        self.block3_conv3 = torch.nn.Conv2d(64, 64, kernel_size=(3,3), padding=(1,1), padding_mode='zeros', stride=1)
        self.block3_conv4 = torch.nn.Conv2d(64, 64, kernel_size=(3,3), padding=(1,1), padding_mode='zeros', stride=2)
        # block up 3
        self.block3_up1 = torch.nn.ConvTranspose2d(64, 64, kernel_size=(3,3), padding=(1,1), padding_mode='zeros', stride=2, output_padding=1)
        self.block3_up2 = torch.nn.Conv2d(64, 64, kernel_size=(3,3), padding=(1,1), padding_mode='zeros', stride=1)
        # block up 2
        self.block2_up1 = torch.nn.ConvTranspose2d(64, 64, kernel_size=(3,3), padding=(1,1), padding_mode='zeros', stride=2, output_padding=1)
        self.block2_up2 = torch.nn.Conv2d(64, 64, kernel_size=(3,3), padding=(1,1), padding_mode='zeros', stride=1)
        # block up 1
        self.block1_up1 = torch.nn.ConvTranspose2d(64, 64, kernel_size=(3,3), padding=(1,1), padding_mode='zeros', stride=2, output_padding=1)
        self.block1_up2 = torch.nn.Conv2d(64, 64, kernel_size=(3,3), padding=(1,1), padding_mode='zeros', stride=1)
        # output
        self.conv_output = torch.nn.Conv2d(64, 3, kernel_size=(1,1), padding=(0,0), padding_mode='zeros', stride=1)
        #
        self.relu = torch.nn.ReLU()

    def forward(self, x, alpha):

        b0 = torch.cat([x, alpha[:,None,None,None].repeat(1, 3, 128, 128)], dim=1)

        b1_c1 = self.relu(self.block1_conv1(b0))
        b1_c2 = self.relu(self.block1_conv2(b1_c1))

        b2_c1 = self.relu(self.block2_conv1(b1_c2))
        b2_c2 = self.relu(self.block2_conv2(b2_c1))

        b3_c1 = self.relu(self.block3_conv1(b2_c2))
        b3_c2 = self.relu(self.block3_conv2(b3_c1))
        b3_c3 = self.relu(self.block3_conv3(b3_c2)) + b3_c1
        b3_c4 = self.relu(self.block3_conv4(b3_c3))

        u2_c1 = self.relu(self.block3_up1(b3_c4)) + b3_c3
        u2_c2 = self.relu(self.block3_up2(u2_c1)) + b2_c2

        u1_c1 = self.relu(self.block2_up1(u2_c2)) + b1_c2
        u1_c2 = self.relu(self.block2_up2(u1_c1))

        u0_c1 = self.relu(self.block1_up1(u1_c2)) + b1_c1
        u0_c2 = self.relu(self.block1_up2(u0_c1))

        output = self.conv_output(u0_c2)

        return output

In [7]:

D = Unet().to('cuda')
optimizer_D = torch.optim.Adam(D.parameters(), lr=0.0005)


### Entrenamiento y evaluación 

Generamos las listas para las metricas 

In [8]:
metrics= defaultdict(list)
train_loss=[]
train_mse=[]
train_psnr=[]
train_ssim=[]

eval_loss=[]
eval_psnr_final=[]
eval_mse=[]
eval_mse_final=[]
eval_psnr=[]
eval_ssim=[]

In [9]:
import torch

def generate_gaussian_noise(tensor, mean=0.0, std=1.0):
    """
    Genera un tensor de ruido gaussiano con las mismas dimensiones que el tensor de entrada.

    Args:
        tensor (torch.Tensor): El tensor de entrada para tomar sus dimensiones.
        mean (float, opcional): Media de la distribución gaussiana. Por defecto es 0.0.
        std (float, opcional): Desviación estándar de la distribución gaussiana. Por defecto es 1.0.

    Returns:
        torch.Tensor: Un tensor de ruido gaussiano con las mismas dimensiones que el tensor de entrada.
    """
    return torch.normal(mean=mean, std=std, size=tensor.size())

# Ejemplo de uso:
# Suponiendo que tienes un tensor de entrada de dimensiones [3, 224, 224]
input_tensor = torch.randn(3, 224, 224)  # Un tensor de ejemplo
noise_tensor = generate_gaussian_noise(input_tensor, mean=0.0, std=1.0)

print(noise_tensor.shape)  # Verifica que las dimensiones coincidan


torch.Size([3, 224, 224])


In [ ]:

# training loop
for period in range(600):
    D.train()
    for batch in tqdm(dataloader, "period " + str(period)):

        # get data      
        mnistc = -1 + 2*batch[0].to("cuda")   #normaliza y trasporta a [-1,1] los valors de las imágenes;

        
        mnisto = generate_gaussian_noise(batch, mean=0.0, std=0.1).to("cuda")   #normaliza y trasporta a [-1,1] los valors de las imágenes;

        x_0 = mnisto
        x_1 = mnistc            
        alpha = torch.rand(batchsize, device="cuda")
        x_alpha = (1-alpha[:,None,None,None]) * x_0 + alpha[:,None,None,None] * x_1

        #perdida 
        loss = torch.sum( (D(x_alpha, alpha) - (x_1-x_0))**2 )


        #MSE

        trainmse = torch.mean( (D(x_alpha, alpha) - (x_1-x_0))**2 )

        #PSNR de este mse
        trainpsnr = 20 * torch.log10(1. / torch.sqrt(trainmse))


        optimizer_D.zero_grad()
        loss.backward()
        optimizer_D.step()
    
        train_loss.append(loss.item())
        train_mse.append(trainmse.item()) 
        train_psnr.append(trainpsnr.item())  

    torch.save({
            'model_state_dict': D.state_dict(),
            'optimizer_state_dict': optimizer_D.state_dict(),
            }, "Bruno600_5.pt")

Esta linea solo hay que ejecutarla en el caso de que se pare el entrenamiento, hay que ponerlo antes del entrenamiento y cambiar el numero de epocas que faltan 

In [10]:
checkpoint = torch.load("Pesos_Difusion/Bruno600_4.pt")
D.load_state_dict(checkpoint['model_state_dict'])
optimizer_D.load_state_dict(checkpoint['optimizer_state_dict'])

/tmp/ipykernel_316583/3626855355.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("Pesos_Difusion/Bruno600_4.pt")


### Sampleo de imágenes de test 


Leeemos las fotos desde la carpeta de test 

In [12]:

# Leer la imagenes con OpenCV, solo sirve para ver el tamaño de la imagen esto
fototest = cv2.imread('/home/brulon/Documentos/TFG/CloudCover/Datasets/AllSky/test/0/C002_20190101_1525.jpg')

# Convertir de BGR a RGB
fototestrgb = cv2.cvtColor(fototest, cv2.COLOR_BGR2RGB)

# Convertir a formato PIL (Pillow), que es compatible con PyTorch
fototestpil = Image.fromarray(fototestrgb)

to_tensor = transforms.ToTensor()

# Aplicar la transformación para convertir la imagen en un tensor de PyTorch
fototest_tensor = to_tensor(fototestrgb)

Y hacemos un sampleo en 10 pasos para visualizar la imagen 

In [ ]:
# # starting points x_alpha = x_0
mnisto = -1 + 2*fototest_tensor.to("cuda")   #normaliza y trasporta a [-1,1] los valors de las imágenes;
# #mnisto = torch.nn.functional.interpolate(mnisto, size=(32,32), mode='bilinear', align_corners=False) 

noise=generate_gaussian_noise(mnisto, mean=0.0, std=0.1).to("cuda")  



x_0 = noise
x_alpha = x_0.unsqueeze(0) 


T = 300
for t in tqdm(range(T), "sampling testing loop"):
    
    #current alpha value
    alpha = t / T * torch.ones(1, device="cuda")

    #update 
    x_alpha = x_alpha + 1/T * D(x_alpha, alpha)
    x_alphashow=x_alpha


    #create result image
    result = np.zeros((1*400, 1*600, 3))         
    #tmp =x_alphashow.detach().cpu().clone().permute(1,2,0).numpy()
    


            
    tmp = 0.5+0.5*x_alphashow.detach().cpu().clone().squeeze(0).permute(1,2,0).numpy()
    result=tmp
    
    #tmp = np.swapaxes(tmp, 0, 2)
    #tmp = np.swapaxes(tmp, 0, 1)
    
    #print(tmp.shape)
    saveImage('Bruno_'+str(t)+'.png', result)

In [ ]:
mnisto = -1 + 2*fototest_tensor.to("cuda")   #normaliza y trasporta a [-1,1] los valors de las imágenes;

for i in tqdm(range(9000), 'fotos generadas'):
    noise=generate_gaussian_noise(mnisto, mean=0.0, std=0.1).to("cuda")  

    x_0 = noise
    x_alpha = x_0.unsqueeze(0) 


    T = 300
    for t in tqdm(range(T), "sampling testing loop"):
        
        #current alpha value
        alpha = t / T * torch.ones(1, device="cuda")

        #update 
        x_alpha = x_alpha + 1/T * D(x_alpha, alpha)
        x_alphashow=x_alpha


        #create result image
        result = np.zeros((1*400, 1*600, 3))         
        #tmp =x_alphashow.detach().cpu().clone().permute(1,2,0).numpy()
    
                
        tmp = 0.5+0.5*x_alphashow.detach().cpu().clone().squeeze(0).permute(1,2,0).numpy()
        result=tmp
        
        #tmp = np.swapaxes(tmp, 0, 2)
        #tmp = np.swapaxes(tmp, 0, 1)
        
        #print(tmp.shape)
        if t==299:
            saveImage('syntetica5/Clase5Sinte_'+str(i)+'.png', result)